# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [ ]:
import numpy as np
import xlwings as xw
import pandas as pd
from pprint import pprint as pp


1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [ ]:
wb = xw.Book('./data/data/себестоимостьА_в1.xlsx')

In [ ]:
sheet = wb.sheets['Рецептура']

In [ ]:
consumption = sheet.range('G7:O10').options(np.array).value
consumption

In [ ]:
unit_price = sheet.range('G14:O14').options(np.array).value
unit_price

In [ ]:
res = np.nan_to_num(consumption * unit_price).sum(axis=1)
res

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [ ]:
sheet.range('T7:T10').options(transpose=True).value = res

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [ ]:
sheet.range('T6').value = 'Себестоимость'
sheet.range('T4:T6').api.Merge()
sheet.range('T4:T6').color = (255, 0, 255)

4. Выполнить 3 с помощью "протягиваемых" формул.

In [ ]:
from xlwings.constants import AutoFillType

In [ ]:
sheet.range('V7').formula = '=SUMPRODUCT(G7:O7, $G$14:$O$14)'
sheet.range('V7').api.AutoFill(sheet.range('V7:V10').api,
                               AutoFillType.xlFillDefault)

## Лабораторная работа 7

In [68]:
import numpy as np
import xlwings as xw
import pandas as pd
from pprint import pprint as pp
from xlwings.constants import AutoFillType

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [69]:
reviews_sample = pd.read_csv("./data/data/reviews_sample.csv", index_col=0, parse_dates=["date"])
recipes_sample_with_tags_ingredients = \
    pd.read_csv("./data/data/recipes_sample_with_tags_ingredients.csv", parse_dates=["submitted"])\
        .drop(columns=["contributor_id", "n_steps", "tags", "n_tags", "recipe_id", "ingredients"])

In [58]:
reviews_sample.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [59]:
recipes_sample_with_tags_ingredients.head()

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,3
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,9
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,12


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц.

In [60]:
from openpyxl import load_workbook

writer = pd.ExcelWriter("./data/data/recipes.xlsx")
reviews_sample.sample(frac=0.05).to_excel(writer, sheet_name = 'Отзывы')
recipes_sample_with_tags_ingredients.sample(frac=0.05).to_excel(writer, sheet_name = 'Рецепты')
writer.save()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [70]:
wb = xw.Book('./data/data/recipes.xlsx')
wb_recipes_sheet = wb.sheets['Рецепты']
wb_reviews_sheet = wb.sheets['Отзывы']
# wb.close()

In [71]:
wb_recipes_sheet.range("H1").value = "seconds_assign"
wb_recipes_sheet.range("H2:H1501").options(transpose=True).value = np.array(wb_recipes_sheet.range("D2:D1501").value) * 60

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [72]:
wb_recipes_sheet.range("I1").value = "seconds_formula"
wb_recipes_sheet.range("I2").formula = "=D2*60"
wb_recipes_sheet.range('I2').api.AutoFill(wb_recipes_sheet.range('I2:I1501').api,
                               AutoFillType.xlFillDefault)

True

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [73]:
wb_recipes_sheet.range("J1").value = "n_reviews"
wb_recipes_sheet.range("J2").formula = '=COUNTIF(Отзывы!C:C, "="&C2)'
wb_recipes_sheet.range('J2').api.AutoFill(wb_recipes_sheet.range('J2:J1501').api,
                               AutoFillType.xlFillDefault)
# wb.close()

True

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [74]:
wb_recipes_sheet['H1:J1'].api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
wb_recipes_sheet['H1:J1'].font.bold = True

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [37]:
for i in wb_recipes_sheet.range("D2:D1501"):
    match i.value:
        case 5:
            i.color = (0, 255, 0)
        case range(5, 10):
            i.color = (255, 255, 0)
        case 5:
            i.color = (255, 0, 0)

In [75]:
for i in wb_recipes_sheet.range("D2:D1501"):
    if i.value < 5:
        i.color = (0, 255, 0)
    elif 5 <= i.value <= 10:
        i.color = (255, 255, 0)
    elif i.value > 10:
        i.color = (255, 0, 0)

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`

В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate():
    wb = xw.Book('./data/data/recipes.xlsx')
    wb_recipes_sheet = wb.sheets['Рецепты']
    wb_reviews_sheet = wb.sheets['Отзывы']

    for i in wb_reviews_sheet:
        print(i)

    wb.close()

In [46]:
# wb = xw.Book('./data/data/recipes.xlsx')
# wb_recipes_sheet = wb.sheets['Рецепты']
# wb_reviews_sheet = wb.sheets['Отзывы']

recipe_ids = np.array(wb_recipes_sheet.range("C2:C1501").value)
for i in wb_reviews_sheet.range("E2:E1501"):
    if i.value not in range(0, 6) and


5.0


In [ ]:
recipe_ids = np.array(wb_recipes_sheet.range("C2:C1501").value)
rating = np.array(wb_recipes_sheet.range("C2:C1501").value)

In [54]:
recipe_ids = np.array(wb_recipes_sheet.range("C2:C1501").value)

array([ 22856., 353624., 270217., ..., 185243., 371688.,  80162.])

In [ ]:
# for i in wb_reviews_sheet.range("E2:E1501"), j in wb_reviews_sheet.range("E2:E1501"):

In [76]:
wb.save("./data/data/recipes.xlsx")
wb.close()